In [ ]:
"""
This module is used to insert csv file to the database
"""

import pandas as pd
from datetime import datetime
from models import * 
from mapping_enums import class_mapping, college_mapping, concentration_mapping, role_mapping, hsr_mapping

In [ ]:
#### MAIN ####
# Import csv file for students' data
df = pd.read_csv(r'/home/khanh/Documents/capstone-hub/backend/M22_Capstone_descriptions.csv')

#### Process NA values ####
NA_VALUE = "NONE"

# print("BEFORE PROCESSING NAs")
# print(f"Number of NA values: {df.isnull().sum()}")

# df = df.fillna(NA_VALUE)

# print("AFTER PROCESSING NAs")
# print(f"Number of NA values: {df.isnull().sum()}")

enum_cols = {
    "role": role_mapping,
    "primary_major": college_mapping,
    "secondary_major": college_mapping,
    "primary_concentration": concentration_mapping,
    "secondary_concentration": concentration_mapping,
    "minor": college_mapping,
    "minor_concentration": concentration_mapping,
    "hsr_review": hsr_mapping
    }


# Function to create lists of data to inject to Models
"""
Take data and put to dictionary
If column is enum:
    Get equivalent enum data -> Put to tables

"""
def create_list_of_data(data, data_cols, model_cols, default_cols=None, default_vals=None):
    """
    Input:
    dataframe data: Source of data
    list[str] data_cols: Column names from the df data
    list[str] model_cols: Column names from the db model
    list[str] default_cols: Default columns
    list[str] default_vals: Default values for the default columns

    Output:
    list[dict] table_to_insert: Table to insert to database
    """
    n = len(data_cols)
    table_to_insert = []

    for index, row in data.iterrows():
        row_data = dict()
        for i in range(n):
            model_col = model_cols[i]
            data_col = data_cols[i]
            row_value = row[data_col]
            if model_col in enum_cols:
                # Map values from data_col to enum_mapping
                # Then put values from enum_mapping to the row data
                mapper = enum_cols[model_col] # Name of the mapper
                key = row_value # Value of the row aka key of the mapper
                if key in mapper:
                    value = mapper[key] # Value to insert aka value of the mapper
                # elif key == NA_VALUE:
                #     value = ""
                else:
                    value = key
                row_data[model_col] = value
            elif data_col == "timestamp":
                datetime_object = datetime.strptime(row_value, '%m/%d/%Y %H:%M:%S')
                row_data[model_col] = datetime_object
            else:
                # Do normal stuffs
                row_data[model_col] = row_value
        if default_cols:
            for i in range(len(default_cols)):
                row_data[default_cols[i]] = default_vals[i]
        table_to_insert.append(row_data)

    return table_to_insert


In [ ]:
# Logins model
logins_cols = ["id", "public_id", "email"]
logins_data_cols = ["id", "id","email"]
logins_default_cols = ["password"]
logins_default_vals = ["Minerva22"]
logins_to_add = create_list_of_data(\
    data=df,\
    data_cols=logins_data_cols,\
    model_cols=logins_cols,\
    default_cols=logins_default_cols,\
    default_vals=logins_default_vals\
    )

# Users model
users_cols = ["id", "login_id", "name",\
            "primary_major", "secondary_major",\
            "primary_concentration", "secondary_concentration",\
            "special_concentration", "minor", "minor_concentration"]
users_data_cols = ["id", "id", "name",\
                "primary_major", "second_major",\
                "primary_concentration", "second_concentration",\
                "special_concentration", "minor", "minor_concentration"]
users_default_cols = ["role", "class_year"]
users_default_vals = [RoleEnum.student, ClassEnum.M22]
users_to_add = create_list_of_data(\
    data=df,\
    data_cols=users_data_cols,\
    model_cols=users_cols,\
    default_cols=users_default_cols,\
    default_vals=users_default_vals\
    )

print(users_to_add[:1])
print(len(users_to_add))


# Projects model
projects_cols = ["id", "user_id", "title", "abstract",\
                "keywords", "feature", "hsr_review", "skills",\
                "los", "custom_los", "advisor", "skills_offering",\
                "skills_requesting", "location", "last_updated"]
projects_data_cols = ["id", "id", "title", "abstract",\
                "keywords", "features", "hsr_status", "skills",\
                "los", "custom_los", "advisor", "skills_offering",\
                "skills_requesting", "location", "timestamp"]
projects_to_add = create_list_of_data(\
    data=df,\
    data_cols=projects_data_cols,\
    model_cols=projects_cols,\
    )

In [ ]:
# Working model
# Data for User model
Users_to_add = [{
    'id': 1,
    'login_id': 1,
    'firstname': 'Anna',
    'lastname': 'Shallat',
    'role': RoleEnum.student,
    'primary_major': CollegeEnum.AH,
    # 'secondary_major': '',
    'primary_concentration': ConcentrationEnum.AL,
    'secondary_concentration': ConcentrationEnum.PEL,
    # 'special_concentration': '',
    # 'minor': '',
    # 'minor_concentration': '',
}, {
    'id': 2,
    'login_id': 2,
    'firstname': 'Dragon',
    'lastname': 'Cruz-Yen',
    'role': RoleEnum.student,
    'primary_major': CollegeEnum.AH,
    # 'secondary_major': '',
    'primary_concentration': ConcentrationEnum.AL,
    # 'secondary_concentration': '',
    # 'special_concentration': '',
    'minor': CollegeEnum.SS,
    'minor_concentration': ConcentrationEnum.CBB,
}, {
    'id': 3,
    'login_id': 3,
    'firstname': 'Elisha',
    'lastname': 'Somasundram',
    'role': RoleEnum.student,
    'primary_major': CollegeEnum.AH,
    # 'secondary_major': '',
    'primary_concentration': ConcentrationEnum.AL,
    'secondary_concentration': ConcentrationEnum.PEL,
    # 'special_concentration': '',
    'minor': CollegeEnum.SS,
    'minor_concentration': ConcentrationEnum.CBB,
}, {
    'id': 4,
    'login_id': 4,
    'firstname': 'Tessa',
    'lastname': 'Heemskerk',
    'role': RoleEnum.student,
    'primary_major': CollegeEnum.AH,
    # 'secondary_major': '',
    'primary_concentration': ConcentrationEnum.AL,
    # 'secondary_concentration': '',
    # 'special_concentration': '',
    'minor': CollegeEnum.BS,
    # 'minor_concentration': '',
}, {
    'id': 5,
    'login_id': 5,
    'firstname': 'Abigail',
    'lastname': 'Gust',
    'role': RoleEnum.student,
    'primary_major': CollegeEnum.AH,
    # 'secondary_major': '',
    'primary_concentration': ConcentrationEnum.HAN,
    # 'secondary_concentration': '',
    # 'special_concentration': '',
    # 'minor': '',
    # 'minor_concentration': '',
}, {
    'id': 6,
    'login_id': 6,
    'firstname': 'Alya',
    'lastname': 'Luk',
    'role': RoleEnum.student,
    'primary_major': CollegeEnum.AH,
    'secondary_major': CollegeEnum.AH,
    'primary_concentration': ConcentrationEnum.HAP,
    'secondary_concentration': ConcentrationEnum.PEL,
    # 'special_concentration': '',
    'minor': CollegeEnum.CS,
    'minor_concentration': ConcentrationEnum.CSAI,
}]


In [ ]:
print(Users_to_add[0])

In [ ]:
df.head(3)

In [7]:
"""
This module is used to insert csv file to the database
"""

import pandas as pd
from datetime import datetime
from models import * 
from mapping_enums import class_mapping, college_mapping, concentration_mapping, role_mapping, hsr_mapping

In [8]:
# Import csv file for students' data
df = pd.read_csv(r'/home/khanh/Documents/capstone-hub/backend/M22_Capstone_descriptions.csv')

In [ ]:
NA_VALUE = "NA"

# print("BEFORE PROCESSING NAs")
# print(f"Number of NA values: {df.isnull().sum()}")

df = df.fillna(NA_VALUE)

In [13]:

enum_cols = {
    "role": role_mapping,
    "primary_major": college_mapping,
    "secondary_major": college_mapping,
    "primary_concentration": concentration_mapping,
    "secondary_concentration": concentration_mapping,
    "minor": college_mapping,
    "minor_concentration": concentration_mapping,
    "hsr_review": hsr_mapping
    }


# Function to create lists of data to inject to Models
"""
Take data and put to dictionary
If column is enum:
    Get equivalent enum data -> Put to tables

"""
def create_list_of_data(data, data_cols, model_cols, default_cols=None, default_vals=None):
    """
    Input:
    dataframe data: Source of data
    list[str] data_cols: Column names from the df data
    list[str] model_cols: Column names from the db model
    list[str] default_cols: Default columns
    list[str] default_vals: Default values for the default columns

    Output:
    list[dict] table_to_insert: Table to insert to database
    """
    n = len(data_cols)
    table_to_insert = []

    for index, row in data.iterrows():
        row_data = dict()
        for i in range(n):
            model_col = model_cols[i]
            data_col = data_cols[i]
            row_value = row[data_col]
            if model_col in enum_cols:
                # Map values from data_col to enum_mapping
                # Then put values from enum_mapping to the row data
                mapper = enum_cols[model_col] # Name of the mapper
                key = row_value # Value of the row aka key of the mapper
                if key in mapper:
                    value = mapper[key] # Value to insert aka value of the mapper
                # elif key == NA_VALUE:
                #     value = mapper
                else:
                    value = key
                row_data[model_col] = value
            elif data_col == "timestamp":
                datetime_object = datetime.strptime(row_value, '%m/%d/%Y %H:%M:%S')
                row_data[model_col] = datetime_object
            else:
                # Do normal stuffs
                row_data[model_col] = row_value
        if default_cols:
            for i in range(len(default_cols)):
                row_data[default_cols[i]] = default_vals[i]
        table_to_insert.append(row_data)

    return table_to_insert

In [14]:

# Logins model
logins_cols = ["id", "public_id", "email"]
logins_data_cols = ["id", "id","email"]
logins_default_cols = ["password"]
logins_default_vals = ["Minerva22"]
logins_to_add = create_list_of_data(\
    data=df,\
    data_cols=logins_data_cols,\
    model_cols=logins_cols,\
    default_cols=logins_default_cols,\
    default_vals=logins_default_vals\
    )

# Users model
users_cols = ["id", "login_id", "name",\
            "primary_major", "secondary_major",\
            "primary_concentration", "secondary_concentration",\
            "special_concentration", "minor", "minor_concentration"]
users_data_cols = ["id", "id", "name",\
                "primary_major", "second_major",\
                "primary_concentration", "second_concentration",\
                "special_concentration", "minor", "minor_concentration"]
users_default_cols = ["role", "class_year"]
users_default_vals = [RoleEnum.student, ClassEnum.M22]
users_to_add = create_list_of_data(\
    data=df,\
    data_cols=users_data_cols,\
    model_cols=users_cols,\
    default_cols=users_default_cols,\
    default_vals=users_default_vals\
    )

print(users_to_add[:1])
print(len(users_to_add))


# Projects model
projects_cols = ["id", "user_id", "title", "abstract",\
                "keywords", "feature", "hsr_review", "skills",\
                "los", "custom_los", "advisor", "skills_offering",\
                "skills_requesting", "location", "last_updated"]
projects_data_cols = ["id", "id", "title", "abstract",\
                "keywords", "features", "hsr_status", "skills",\
                "los", "custom_los", "advisor", "skills_offering",\
                "skills_requesting", "location", "timestamp"]
projects_to_add = create_list_of_data(\
    data=df,\
    data_cols=projects_data_cols,\
    model_cols=projects_cols,\
    )

# print(projects_to_add[:3])
# print(len(projects_to_add))


def insert_data(dict_to_add, table):
    """To insert data"""
    # import sqlite3

    # db = sqlite3.connect("database.db")
    # cur = db.cursor()


    # Create application context to add data to the database
    from app import create_app
    my_app = create_app()
    my_app.app_context().push()

    # List of keys with (list_of_data, model) to iterate
    # keys = [(logins_to_add, Login), (users_to_add, User), (projects_to_add, Project)]
    # Insert data
    # for dict_to_add, table in keys:
    for dict_row in dict_to_add:
        try:
            stmt = table(**dict_row)
            db.session.add(stmt)
            db.session.commit()
        except Exception as e:
            db.session.rollback()
            print(e)
            continue
        finally:
            db.session.close()

[{'id': 1, 'login_id': 1, 'name': 'Vick Boyu Liu', 'primary_major': <CollegeEnum.AH: 'Arts & Humanities'>, 'secondary_major': <CollegeEnum.BS: 'Business'>, 'primary_concentration': <ConcentrationEnum.AL: 'Arts and Literature'>, 'secondary_concentration': <ConcentrationEnum.NBV: 'New Business Ventures'>, 'special_concentration': nan, 'minor': nan, 'minor_concentration': nan, 'role': <RoleEnum.student: 'Student'>, 'class_year': <ClassEnum.M22: 'Class of 2022'>}]
143


In [11]:
insert_data(logins_to_add, Login)

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "logins_pkey"
DETAIL:  Key (id)=(1) already exists.

[SQL: INSERT INTO logins (id, public_id, email, password) VALUES (%(id)s, %(public_id)s, %(email)s, %(password)s)]
[parameters: {'id': 1, 'public_id': 1, 'email': 'vick_liu@uni.minerva.edu', 'password': 'Minerva22'}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)
(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "logins_pkey"
DETAIL:  Key (id)=(2) already exists.

[SQL: INSERT INTO logins (id, public_id, email, password) VALUES (%(id)s, %(public_id)s, %(email)s, %(password)s)]
[parameters: {'id': 2, 'public_id': 2, 'email': 'gabrielle.vonseggern@uni.minerva.edu', 'password': 'Minerva22'}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)
(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "logins_pkey"
DETAIL:  Key (id)=(3) already exists.

[SQL: INSERT INTO logins (id, 

In [12]:
insert_data(users_to_add, User)

(builtins.LookupError) 'nan' is not among the defined enum values. Enum name: collegeenum. Possible values: CS, BS, NS, ..., SS
[SQL: INSERT INTO users (id, login_id, name, role, class_year, primary_major, secondary_major, primary_concentration, secondary_concentration, special_concentration, minor, minor_concentration) VALUES (%(id)s, %(login_id)s, %(name)s, %(role)s, %(class_year)s, %(primary_major)s, %(secondary_major)s, %(primary_concentration)s, %(secondary_concentration)s, %(special_concentration)s, %(minor)s, %(minor_concentration)s)]
[parameters: [{'minor_concentration': nan, 'secondary_concentration': <ConcentrationEnum.NBV: 'New Business Ventures'>, 'id': 1, 'minor': nan, 'role': <RoleEnum.st ... (164 characters truncated) ... _major': <CollegeEnum.AH: 'Arts & Humanities'>, 'special_concentration': nan, 'primary_concentration': <ConcentrationEnum.AL: 'Arts and Literature'>}]]
(builtins.LookupError) 'nan' is not among the defined enum values. Enum name: collegeenum. Possible v

In [ ]:
insert_data(projects_to_add, Project)

In [16]:
df.head(3)

,timestamp,id,email,name,primary_major,primary_concentration,second_major,second_concentration,special_concentration,minor,...,keywords,features,hsr_status,skills,los,custom_los,advisor,skills_offering,skills_requesting,location
0,9/11/2021 8:05:28,1,vick_liu@uni.minerva.edu,Vick Boyu Liu,Arts & Humanities,Arts and Literature,Business,New Business Ventures,NaN,NaN,...,"Theater, Historical Trauma, Best Practices, So...","Creative (e.g., writing, art), Secondary resea...",N/A (project does NOT involve interacting with...,"Music theory, basic song writing and theatre p...",#ah166-produceArt\n#b144-Implementation\n#ah11...,#playwriting\n#PitchDeck,Kamler,"Comics, Musical",Art producing and music theory,London (BST then GMT/UTC)
1,9/18/2021 9:05:17,2,gabrielle.vonseggern@uni.minerva.edu,Gabrielle Von Seggern,Arts & Humanities,Arts and Literature,Social Sciences,"Cognition, Brain, and Behavior",NaN,NaN,...,"Mythology, ancient cultures, creative, illustr...","Creative (e.g., writing, art), Secondary resea...",N/A (project does NOT involve interacting with...,Film theory (especially visual theories and us...,"#ah110primarysources, #ah110historicalperspect...",NaN,Fiorelli,NaN,NaN,Berlin (GMT+2)
2,9/16/2021 9:59:12,3,arshallat@uni.minerva.edu,Anna Shallat,Arts & Humanities,Arts and Literature,NaN,"Philosophy, Ethics, and the Law",NaN,NaN,...,"Zines, Ethics, LGBTQ+, subculture","Creative (e.g., writing, art), Secondary resea...",N/A (project does NOT involve interacting with...,- illustration technique\n- creative writing\n...,"#ah144ethicaltheory,\n#ah146rupturesanddisloca...",NaN,Bogucki,"Dance, storytelling, humor, illustration, text...","education, sketching/painting, animation/video...",Peoria (CST)
